<a href="https://colab.research.google.com/github/yongsa-nut/TU_CN409_GenAI_67_2/blob/main/DB_Agentic_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DuckDB Agentic Demo

Adapted from https://github.com/disler/single-file-agents/blob/main/sfa_duckdb_anthropic_v2.py

In [ ]:
!pip install anthropic

In [50]:
import duckdb
import subprocess
import json
from typing import List
from rich import print
from anthropic import Anthropic
from google.colab import userdata

In [ ]:
#download the data
!wget https://github.com/disler/single-file-agents/raw/refs/heads/main/data/analytics.db

In [18]:
# Connect to the database
DB_PATH = '/content/analytics.db'
DB_CONN = duckdb.connect(DB_PATH)

## Database Agent

5 tools (functions)
- `list_table`: Returns list of available tables in database
- `describe_table`: Returns schema info for specified table
- `sample_table`: Returns sample rows from specified table, always specify row_sample_size
- `run_test_sql_query`: Tests a SQL query and returns results (only visible to agent)
- `run_final_sql_query`: Runs the final validated SQL query and shows results to user

The agent will keep using tools looking at the result and keep going until it generates accurate queries for the user.

### `list_tables`

Returns a list of tables in the database. The agent uses this to discover available tables and make informed decisions.

In [19]:
def list_tables(reasoning: str) -> List[str]:
    """Returns a list of tables in the database.

    The agent uses this to discover available tables and make informed decisions.

    Args:
        reasoning: Explanation of why we're listing tables relative to user request

    Returns:
        List of table names as strings
    """
    try:
        # Use the global connection
        result = DB_CONN.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

        # Extract table names from the result
        table_names = [row[0] for row in result]

        print(f"List Tables Tool - Reasoning: {reasoning}")
        return table_names
    except Exception as e:
        print(f"Error listing tables: {str(e)}")
        return []

In [29]:
list_tables('Test')

List Tables Tool - Reasoning: Test

['User']

### `describe_table`

Returns schema information about the specified table.
The agent uses this to understand table structure and available columns.

In [30]:
def describe_table(reasoning: str, table_name: str) -> str:
    """Returns schema information about the specified table.

    The agent uses this to understand table structure and available columns.

    Args:
        reasoning: Explanation of why we're describing this table
        table_name: Name of table to describe

    Returns:
        String containing table schema information
    """
    try:
        # Use the global connection to execute DESCRIBE
        result = DB_CONN.execute(f"DESCRIBE {table_name}").fetchall()

        # Convert result to a string - DuckDB's DESCRIBE already returns
        # nicely formatted column information
        schema_info = "\n".join(str(row) for row in result)

        # Log the operation
        print(f"Describe Table Tool - Table: {table_name} - Reasoning: {reasoning}")
        return schema_info

    except Exception as e:
        print(f"Error describing table: {str(e)}")
        return ""

In [31]:
print(describe_table('Test','User'))

Describe Table Tool - Table: User - Reasoning: Test

('id', 'UUID', 'YES', None, None, None)
('name', 'VARCHAR', 'YES', None, None, None)
('age', 'BIGINT', 'YES', None, None, None)
('city', 'VARCHAR', 'YES', None, None, None)
('score', 'DOUBLE', 'YES', None, None, None)
('is_active', 'BOOLEAN', 'YES', None, None, None)
('status', 'VARCHAR', 'YES', None, None, None)
('created_at', 'DATE', 'YES', None, None, None)

```
('id', 'UUID', 'YES', None, None, None)
 │      │      │      │     │     │
 │      │      │      │     │     └─ 6. Column comment or extra info (None here)
 │      │      │      │     └───── 5. Column position or other metadata (None here)
 │      │      │      └─────────── 4. Default value (None here)
 │      │      └──────────────────── 3. Nullable status ('YES' means column can be NULL)
 │      └─────────────────────────── 2. Data type (UUID type)
 └────────────────────────────────── 1. Column name (id)
```

### `sample_table`

Returns a sample of rows from the specified table.
The agent uses this to understand actual data content and patterns.

In [34]:
def sample_table(reasoning: str, table_name: str, row_sample_size: int) -> str:
    """Returns a sample of rows from the specified table.

    The agent uses this to understand actual data content and patterns.

    Args:
        reasoning: Explanation of why we're sampling this table
        table_name: Name of table to sample from
        row_sample_size: Number of rows to sample aim for 3-5 rows

    Returns:
        String containing sample rows in readable format
    """
    try:
        # Use the global connection to select sample rows
        result = DB_CONN.execute(
            f"SELECT * FROM {table_name} LIMIT {row_sample_size}"
        ).fetchall()

        # Get column names for context
        columns = DB_CONN.execute(
            f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}'"
        ).fetchall()
        column_names = [col[0] for col in columns]

        # Format the output with column names and data
        header = str(column_names)
        rows = [str(row) for row in result]

        sample_data = header + "\n" + "\n".join(rows)

        print(f"Sample Table Tool - Table: {table_name} - Rows: {row_sample_size} - Reasoning: {reasoning}")

        return sample_data

    except Exception as e:
        print(f"Error sampling table: {str(e)}")
        return ""

In [ ]:
print(sample_table('Test','User',5))

### `run_test_sql_query`

Executes a test SQL query and returns results.

The agent uses this to validate queries before finalizing them. Results are only shown to the agent, not the user.

In [38]:
def run_test_sql_query(reasoning: str, sql_query: str) -> str:
    """Executes a test SQL query and returns results.

    The agent uses this to validate queries before finalizing them.
    Results are only shown to the agent, not the user.

    Args:
        reasoning: Explanation of why we're running this test query
        sql_query: The SQL query to test

    Returns:
        Query results as a string
    """
    try:
        # Use the global connection to execute the query
        result = DB_CONN.execute(sql_query).fetchall()

        # Convert result to a simple string representation
        # For educational purposes, keeping the output straightforward
        output = "\n".join(str(row) for row in result)

        print(f"Test Query Tool - Reasoning: {reasoning}")
        print(f"Query: {sql_query}")

        return output
    except Exception as e:
        print(f"Error running test query: {str(e)}")
        return str(e)

In [ ]:
print(run_test_sql_query('Test','SELECT * FROM User WHERE AGE > 50'))

### `run_final_sql_query`

In [43]:
def run_final_sql_query(reasoning: str, sql_query: str) -> str:
    """Executes the final SQL query and returns results to user.

    This is the last tool call the agent should make after validating the query.

    Args:
        reasoning: Final explanation of how this query satisfies user request
        sql_query: The validated SQL query to run

    Returns:
        Query results as a string
    """
    try:
        # Use the global connection to execute the query
        result = DB_CONN.execute(sql_query).fetchall()

        # Convert result to a string - format is a simple representation of each row
        results_str = "\n".join(str(row) for row in result)

        # Use regular print with simple formatting
        print(f"Final Query Tool\nReasoning: {reasoning}\nQuery: {sql_query}")

        return results_str
    except Exception as e:
        print(f"Error running final query: {str(e)}")
        return str(e)

In [ ]:
print(run_final_sql_query('Test','SELECT * FROM User WHERE AGE > 50'))

## Defining Tools

In [45]:
tools=[
        {
            "name": "list_tables",
            "description": "Returns a list of available tables in database",
            "input_schema": {
                "type": "object",
                "properties": {
                    "reasoning": {
                        "type": "string",
                        "description": "Explanation for listing tables",
                    }
                },
                "required": ["reasoning"],
            },
        },
        {
            "name": "describe_table",
            "description": "Returns schema info for a specified table",
            "input_schema": {
                "type": "object",
                "properties": {
                    "reasoning": {
                        "type": "string",
                        "description": "Why we need to describe this table",
                    },
                    "table_name": {
                        "type": "string",
                        "description": "Name of a table to describe",
                    },
                },
                "required": ["reasoning", "table_name"],
            },
        },
        {
            "name": "sample_table",
            "description": "Returns sample rows from specified table",
            "input_schema": {
                "type": "object",
                "properties": {
                    "reasoning": {
                        "type": "string",
                        "description": "Why we need to sample this table",
                    },
                    "table_name": {
                        "type": "string",
                        "description": "Name of table to sample",
                    },
                    "row_sample_size": {
                        "type": "integer",
                        "description": "Number of rows to sample aim for 3-5 rows",
                    },
                },
                "required": ["reasoning", "table_name", "row_sample_size"],
            },
        },
        {
            "name": "run_test_sql_query",
            "description": "Tests a SQL query and returns results (only visible to agent)",
            "input_schema": {
                "type": "object",
                "properties": {
                    "reasoning": {
                        "type": "string",
                        "description": "Why we're testing this specific query",
                    },
                    "sql_query": {
                        "type": "string",
                        "description": "The SQL query to test",
                    },
                },
                "required": ["reasoning", "sql_query"],
            },
        },
        {
            "name": "run_final_sql_query",
            "description": "Runs the final validated SQL query and shows results to user",
            "input_schema": {
                "type": "object",
                "properties": {
                    "reasoning": {
                        "type": "string",
                        "description": "Final explanation of how query satisfies user request",
                    },
                    "sql_query": {
                        "type": "string",
                        "description": "The validated SQL query to run",
                    },
                },
                "required": ["reasoning", "sql_query"],
            },
        },
    ]

In [51]:
# Helper function calling
def call_function(func_name, func_args):

    if func_name == "list_tables":
        result = list_tables(reasoning=func_args["reasoning"])
    elif func_name == "describe_table":
        result = describe_table(
            reasoning=func_args["reasoning"],
            table_name=func_args["table_name"],
        )
    elif func_name == "sample_table":
        result = sample_table(
            reasoning=func_args["reasoning"],
            table_name=func_args["table_name"],
            row_sample_size=func_args["row_sample_size"],
        )
    elif func_name == "run_test_sql_query":
        result = run_test_sql_query(
            reasoning=func_args["reasoning"],
            sql_query=func_args["sql_query"],
        )
    elif func_name == "run_final_sql_query":
        result = run_final_sql_query(
            reasoning=func_args["reasoning"],
            sql_query=func_args["sql_query"],
        )
    else:
        raise Exception(f"Unknown tool call: {func_name}")
    return result

## Prompt

**Note**: When you call the Anthropic API with the tools parameter, we construct a special system prompt from the tool definitions, tool configuration, and any user-specified system prompt. The constructed prompt is designed to instruct the model to use the specified tool(s) and provide the necessary context for the tool to operate properly.

In [63]:
AGENT_PROMPT = """You are a world-class expert at crafting precise DuckDB SQL queries.
Your goal is to generate accurate queries that exactly match the user's data needs.

<instructions>
    - Use the provided tools to explore the database and construct the perfect query.
    - Start by listing tables to understand what's available.
    - Describe tables to understand their schema and columns.
    - Sample tables to see actual data patterns.
    - Test queries before finalizing them.
    - Only call run_final_sql_query when you're confident the query is perfect.
    - Be thorough but efficient with tool usage.
    - If you find your run_test_sql_query tool call returns an error or won't satisfy the user request, try to fix the query or try a different query.
    - Think step by step about what information you need.
    - Be sure to specify every parameter for each tool call.
    - Every tool call should have a reasoning parameter which gives you a place to explain why you are calling the tool.
</instructions>

<user-request>
    {{user_request}}
</user-request>
"""

## Main Agent Loop

In [ ]:
# Initialize Anthropic client
client = Anthropic(api_key=userdata.get('anthropic'))

response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{"role": "user", "content": 'Hello Claude. How are you?'}])
print(response.content[0].text)

Example to test: Show me all users with score above 80

In [65]:
user_query = input('Enter query: ')
completed_prompt = AGENT_PROMPT.replace("{{user_request}}", user_query)
messages = [{"role": "user", "content": completed_prompt}]

max_iteration = 15
compute_iterations = 0
final_result = False

# Main agent loop
while not final_result:
    print(f"\n=== Agent Loop {compute_iterations+1}/{max_iteration} ===")
    compute_iterations += 1

    if compute_iterations > max_iteration:
      print("Warning: Reached maximum compute loops without final query" )
      break

    # Generate content with tool support
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        messages=messages,
        tools=tools,
        tool_choice={"type": "any"}  # Always force a tool call
    )

    # Look for tool calls in the response (expecting ToolUseBlock objects)
    tool_calls = []
    for block in response.content:
        if hasattr(block, "type") and block.type == "tool_use":
            tool_calls.append(block)

    for tool_call in tool_calls:
        tool_use_id = tool_call.id
        func_name = tool_call.name
        func_args = (tool_call.input)

        print(f"Tool Call: {func_name}({json.dumps(func_args)})")
        messages.append({"role": "assistant", "content": response.content})

        try:
            result = call_function(func_name, func_args)
            if func_name == "run_final_sql_query":
                print(f"\n\n***** Final Results *****\n")
                print(result)
                final_result = True
                break
            print(f"Tool Call Result: {func_name}(...) ->\n{result}")

            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": str(result),
                        }
                    ],
                }
            )

        except Exception as e:
            error_msg = f"Error executing {func_name}: {str(e)}"
            print(f"{error_msg}")
            # See info https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#troubleshooting-errors
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": error_msg,
                            "is_error": True
                        }
                    ]
                }
            )
            continue


Enter query: Show me all users with score above 80


=== Agent Loop 1/15 ===

Tool Call: list_tables({"reasoning": "To address the request, I first need to understand what tables are available 
in the database to identify which one might contain user information and scores."})

List Tables Tool - Reasoning: To address the request, I first need to understand what tables are available in the 
database to identify which one might contain user information and scores.

Tool Call Result: list_tables(...) ->
['User']

...

=== Agent Loop 5/15 ===

Tool Call: run_final_sql_query({"reasoning": "The test query successfully returned all users with scores above 80, 
showing 9 users who meet this criteria. The query is correctly filtering on the score column and ordering results 
by score in descending order to show the highest-scoring users first. This matches exactly what the user 
requested.", "sql_query": "SELECT * FROM User WHERE score > 80 ORDER BY score DESC;"})

Final Query Tool
Reasoning: The test query successfully returned all users with scores above 80, showing 9 users who meet this 
criteria. The query is correctly filtering on the score column and ordering results by score in descending order to
show the highest-scoring users first. This matches exactly what the user requested.
Query: SELECT * FROM User WHERE score > 80 ORDER BY score DESC;

***** Final Results *****

(UUID('20b827b3-f183-44db-9f4f-f30b361c8a83'), 'Diana', 63, 'Singapore', 96.18, False, 'inactive', 
datetime.date(2024, 8, 2))
(UUID('94efbf8b-4c95-4feb-9eda-900192276be7'), 'Fiona', 33, 'Singapore', 95.48, True, 'active', datetime.date(2024,
4, 30))
(UUID('66036da1-7704-4ca6-9ec2-1e6d560db610'), 'Alice', 61, 'Singapore', 93.69, True, 'archived', 
datetime.date(2024, 5, 6))
(UUID('2e4aded0-53aa-4668-b3b1-90f4747aa295'), 'Fiona', 20, 'Tokyo', 88.64, True, 'inactive', datetime.date(2023, 
4, 8))
(UUID('8612b8d7-4524-4575-a782-a01c4a0c88a3'), 'Bob', 25, 'Berlin', 88.28, False, 'pending', datetime.date(2025, 1,
16))
(UUID('9c2378d3-f46d-4f19-8e9e-b9053e6e57ea'), 'Charlie', 54, 'Tokyo', 86.24, False, 'archived', 
datetime.date(2023, 11, 15))
(UUID('1666fe7c-a2b1-44d5-abba-293c4e0f9b23'), 'Charlie', 58, 'Sydney', 85.02, True, 'active', datetime.date(2024, 
9, 9))
(UUID('ad992026-c339-40eb-a669-f16983d226ca'), 'Diana', 29, 'Berlin', 84.17, False, 'active', datetime.date(2023, 
7, 26))
(UUID('3674700e-a83f-4450-97c8-04f80f2d2e89'), 'Charlie', 30, 'Sydney', 80.83, False, 'active', datetime.date(2024,
10, 22))